import multiprocessing

def funSquare(num):
    return num ** 2

if __name__ == '__main__':
    pool = multiprocessing.Pool(4)
    results = pool.map(funSquare, range(100))
    print(results)

import multiprocessing
def multi_run_wrapper(args):
   return add(*args)
def add(x,y):
    return x+y
if __name__ == "__main__":
    from multiprocessing import Pool
    pool = Pool(4)
    results = pool.map(multi_run_wrapper,[(1,2),(2,3),(3,4)])
    print(results)
    


In [1]:
import os
def distributed_q( L,N,wn,g,dx,rho_c,delta_rho_c, D, index_lava):
    # update the L matrix base on new wn
    # distributed_q takes the wn as an argument and calculate the updated 
    # wn 
    # update distributed load q(x) in the coefficients based on calculated wn

    #infill_flag get value of 1 at where wn(x)>=0 and get 0 when wn(x)<0
    infill_flag = (wn >= 0);
    infill_flag[N-4:N] = 0; # for boundary condition 1~4
    if(index_lava == 1):
        infill_flag[0:N] = 0 #for lava sea condition
        
    num_i = N;  
    num_j = N - 4;

    coef_i_minus_2 = 1;                         # coefficient for w(i-2) (j == i)
    coef_i_minus_1 = -4;                        # coefficient for w(i-1) (i = j+1)
    coef_i = 6 + delta_rho_c * g / (D / dx**4.);  # coeficient for w(i)   (i = j+2)
    coef_i_wn_positive = 6 + rho_c * g / (D / dx**4.); # coef for wn(x) >= 0
    coef_i_plus_1 = -4.;                         # coefficient for w(i+1) (i = j+3)
    coef_i_plus_2 = 1.;                          # coefficient for w(i+2) (i = j+4)

    for j in range(0,num_j):
        #for i = 1:1:num_i
        for i in range(0,num_i):
            if(i == j):
                L[j,i] = coef_i_minus_2;
            elif(i==j+1):
                L[j,i] = coef_i_minus_1;
            elif(i==j+2):
                if(infill_flag[i,0] == False):
                    L[j,i] = coef_i;
                elif(infill_flag[i,0] == True):
                    L[j,i] = coef_i_wn_positive;  
            elif(i==j+3):
                L[j,i] = coef_i_plus_1;
            elif(i==j+4):
                L[j,i] = coef_i_plus_2;
    return L

In [2]:
# THis python notebook prepare the library for SDR profile 
#due to one load with different Te from 0~20km with 0.1 km increment
import math 
import matplotlib.pyplot as plt
import numpy as np
Te_max = 20000.;
Te_min = 100.;
N_Te = 200;     #number of points for Te

Te_array = np.linspace(Te_min, Te_max, N_Te);              #[m] effective plate thickness
#Hd_array = np.array([5000.,8000.,10000.,13000.,15000.,18000.,20000.]);  #[m] height of the dike
Hd_array = np.array([6000.,12000.,14000.,16000.]);  #[m] height of the dike
N_Hd = len(Hd_array)

list = []
for item in Hd_array:
    for jtem in Te_array:
        if jtem <= item:
            list.append((item,jtem))


In [3]:
list

[(6000.0, 100.0),
 (6000.0, 200.0),
 (6000.0, 300.0),
 (6000.0, 400.0),
 (6000.0, 500.0),
 (6000.0, 600.0),
 (6000.0, 700.0),
 (6000.0, 800.0),
 (6000.0, 900.0),
 (6000.0, 1000.0),
 (6000.0, 1100.0),
 (6000.0, 1200.0),
 (6000.0, 1300.0),
 (6000.0, 1400.0),
 (6000.0, 1500.0),
 (6000.0, 1600.0),
 (6000.0, 1700.0),
 (6000.0, 1800.0),
 (6000.0, 1900.0),
 (6000.0, 2000.0),
 (6000.0, 2100.0),
 (6000.0, 2200.0),
 (6000.0, 2300.0),
 (6000.0, 2400.0),
 (6000.0, 2500.0),
 (6000.0, 2600.0),
 (6000.0, 2700.0),
 (6000.0, 2800.0),
 (6000.0, 2900.0),
 (6000.0, 3000.0),
 (6000.0, 3100.0),
 (6000.0, 3200.0),
 (6000.0, 3300.0),
 (6000.0, 3400.0),
 (6000.0, 3500.0),
 (6000.0, 3600.0),
 (6000.0, 3700.0),
 (6000.0, 3800.0),
 (6000.0, 3900.0),
 (6000.0, 4000.0),
 (6000.0, 4100.0),
 (6000.0, 4200.0),
 (6000.0, 4300.0),
 (6000.0, 4400.0),
 (6000.0, 4500.0),
 (6000.0, 4600.0),
 (6000.0, 4700.0),
 (6000.0, 4800.0),
 (6000.0, 4900.0),
 (6000.0, 5000.0),
 (6000.0, 5100.0),
 (6000.0, 5200.0),
 (6000.0, 5300.0),
 (

In [10]:
def function(Hd,Te):
    # #----------------------------------------------------------------
    # # Initialization
    # #----------------------------------------------------------------
    #Length = 600000;             #[m] total length    # source of error
    Length = 200000;             #[m] total length 
    N = 2001;               # number of node points
    dx = Length / (N - 1);       #[m] distance between node points
    x = np.linspace(0, Length, N);  #[m] initialize the x array
    w = np.zeros((1, N));        #[m] analytic solution to deflection w(x)
    #dw = np.zeros((1, N));       #[m] analytic solution to deflection dw(x)
    #wn = np.zeros((1, N));       #[m] numerical solution to deflection
    #dw = np.zeros((N, 1));       #[m] analytic solution to deflection dw(x)
    wn = np.zeros((N, 1));       #[m] numerical solution to deflection
    
    g = 10.;                 #[m/s**2] gravitational acceleration
    rho_d = 3000.;           #[kg/m**3] density of the solidified dike
    rho_f = 2800.;           #[kg/m**3] density of the fluid dike
    delta_rho_d = rho_d - rho_f;
    rho_i_sedi = 2300.;
    rho_i = 2800.;           #[kg/m**3] density of the infill
    rho_c = 3000.;           #[kg/m**3] density of the underlying lower crust or mantle
    sediment = 0;  # whether infill is sedi or lava  (1 means sedi)
    if sediment == 1:
        delta_rho_c = rho_c - rho_i_sedi;
    else:
        delta_rho_c = rho_c - rho_i;


    E = 7.5 * 10.**10.;          #[Pa] Young's modulus; for lame constants of 30GPa
    mu = 0.25;              # Poisson's ratio
    if Te<= Hd:
        print("Hd={},Te={}".format(Hd,Te))
        D = (E * Te**3.) / (12 * (1 - mu**2.)); #[N*m] Flexural rigidity
        alpha = (4. * D / (delta_rho_c * g))**0.25;#[m] Flexural wavelength
        dV_0 = dx * g * Hd * delta_rho_d; #[N/m] volcanic line load due to denser dike

        #----------------------------------------------------------------
        # Finite difference solution to D * d4w/dx4 + q(x) = 0
        # Implicit solution L * W  = R
        # L are the corresponding coefficients of w(i-2 ~ i+2)
        # W are all the w
        # R are the right hand side which are the extra loads
        #----------------------------------------------------------------
        #----------------------------------------------------------------
        # Lava sea or not
        #----------------------------------------------------------------
        index_lava = 0;
        # index_lava = 1 means lava sea where:
                                # q(x) = (rho_i - rho_c) * g * w(x) for any w(x)
        # index_lava = 0 means:
                                # q(x) = (rho_i - rho_c) * g at w(x) < 0
                                # q(x) = - rho_c * g at w(x) >= 0
        #----------------------------------------------------------------
        # Setup the R matrix (N-5+1+4, 1)  # The added 4 is for 4 BCs
        #----------------------------------------------------------------
        R = np.zeros((N-5+1+4, 1));
        #----------------------------------------------------------------
        # setup the L matrix (coefficients of w)
        # D/dx**4 * (w(i+2) - 4*w(i+1) + 6*w(i) - 4*w(i-1) + w(i-2)) + ...
        # delta_rho_c * g * w(i) = 0
        #----------------------------------------------------------------
        num_i = N-1;  
        num_j = N-1 - 4;
        L = np.zeros((N, N));
        # BC1 approximate line load due to denser dike at the center 
        # R(1) = -dV_0 / 2 / (D / dx**4);
        # Rather than applying on the RHS of the first equation, we use BCs of 
        # V = dV_0 at x = 0

        # The remaining four BCs can be added to the L and R matrices
        # The row number of the four BCs is random and interchangeable
        #---------------------
        # BC1: wn(N) = 0;
        #---------------------
        L[num_j+1,num_i] = 1;
        #---------------------
        # BC2: w'(inf) = 0;
        # wn(N-1) = wn(N);
        #---------------------
        L[num_j+2,num_i] = 1;
        L[num_j+2,num_i-1] = -1;
        #---------------------
        # BC3: M = 0 at x = 0  --> d2w/dx2|(x=0) = 0
        # d2w/dx2 = (wn(i+1) - 2wn(i) + wn(i-1)) / (dx)**2;
        # M = -D * d2w/dx2 = -D * (wn(i+1) - 2wn(i) + wn(i-1)) / (dx)**2;
        #---------------------
        L[num_j+3, 0] = 1;
        L[num_j+3, 1] = -2;
        L[num_j+3, 2] = 1;
        #---------------------
        # BC4 V = dM/dx = -D * d3w/dx3 = V_0/2 = dV_0 at x = 0
        # d3w/dx3|(i-0.5) = (d2w/dx2|i - d2w/dx2|i-1) / dx
        #               = (wn(i+1) -3*wn(i)+3*wn(i-1)-wn(i-2)) / (dx)**3
        # -D * (-wn(1) + 3 * wn(2) - 3 * wn(3) + wn(4)) / dx**3;
        #---------------------
        L[num_j+4, 0] = -1;
        L[num_j+4, 1] = 3;
        L[num_j+4, 2] = -3;         
        L[num_j+4, 3] = 1;
        R[num_j+4,0] = dV_0 / (-D/dx**3); # value at the RHS

        num_iterate = 6;
        #print(num_iterate)
        for i in range(0, num_iterate):
        # update L base on wn   --> Assigning distributed load q(x)
        #print(i)
            L = distributed_q( L,N,wn,g,dx,rho_c,delta_rho_c, D, index_lava); 
            wn = np.linalg.solve(L,R)

        #os.mkdir("Te-library-dx100_test")
        os.chdir("/Volumes/Kelvin_2017/Thesis_Columbia/My_papers/Tian-Buck-2019-JGR-SDR_lithospheric_thickness/1Codes"
                 +"/Python_SDR/SDR_Te_inversion_grid_search/Te-library-dx100_test")
        file_out = open("Hd_{}_Te_{}.txt".format(Hd/1000,Te/1000),"w")
        for i in range(N):
            file_out.write("{},".format(wn[i,0]))
        file_out.close()

In [ ]:
import multiprocessing

def multi_run_wrapper(args):
   return function(*args)

if __name__ == "__main__":
    from multiprocessing import Pool
    pool = Pool(4)
    results = pool.map(multi_run_wrapper,list)


Hd=6000.0,Te=100.0
Hd=12000.0,Te=100.0
Hd=6000.0,Te=3100.0
Hd=12000.0,Te=3100.0
Hd=12000.0,Te=200.0
Hd=6000.0,Te=3200.0
Hd=6000.0,Te=200.0
Hd=12000.0,Te=3200.0
